In [1]:
# ========================================================================= CHAVES PARA CIFRAÇÃO E PARTILHA

# Geração de chaves: BOB
def Elgamalinicial (bits):
    """
Função que invoca a escolha da chave privada e cálculo da chave pública Elgamal.
Input: recebe o número de bits.
Output: retorna um par de chaves, total de participantes ao esquema de partilha, total de 
elementos para recuperação do segredo, o grupo e o seu respetivo gerador.

Uso: Alice define o total de participantes na partilha da chave privada, o número de elemetos 
necessários para a reconstrução do segredo, calcula o p primo, onde Zp* é grupo, escolhe a chave
privada x∈Zp*,g gerador do grupo, calcula y= g^x mod p. A chave a
pública é: (y;g;p).

Exemplo: Suponhamos que pretende-se gerar uma chave de 128 bits,
então: a chave privada: 190422571672400112311789107296726056505,
Chave pública: (224757691300596776616407300685410361353, 3,
65773925495537165919969078067390052602)
    """
    p=random_prime(2**bits)
    Zp=IntegerModRing(p)
    g=Zp.multiplicative_generator()
    n=input("Quantos participantes terá o esquema de partilha da CHAVE PRIVADA?")
    while n==1:
        n=input("Impossivel partilhar a chave. Insira um numero maior que 1. Quantos participantes terá o esquema de partilha da CHAVE PRIVADA?")
    k=input("Quantas partes partes secretas serão necessárias para RECONSTRUIR A CHAVE?")
    while k<=1 or k>n:
        k=input("Insira outro valor de k, menor ou igual que n. Quantas partes partes secretas serão necessárias para RECONSTRUIR A CHAVE?")
    a0= ceil(randint(2, p-1))
    y=g**a0
    PuKey=(p, g, y)
    PrKey=a0
    return PuKey, PrKey, n, k, Zp, g

# ================================================================================================= ASSINATURA

def voto():
    """
Função que invoca a geração de uma mensagem.
Input: nenhum parâmetro é usado.
Output: retorna a mensagem
Uso: o eleitor escolhe um número que representa o concorrente.
    """
    m=input("Digite um numero para eleger o seu CANDIDATO/ PARTIDO:")
    while m<=0:
        m=input("Voto errado. Digite um numero para eleger o seu CANDIDATO/ PARTIDO:")
    return m

#SIGNATARIO

def chavesblindsig (bits):
    """
Função que invoca a geração do par de chaves RSA.
Input: número de bits
Output: chave privada e pública.

Uso: escolhe-se dois números primos p e q, calcula-se a função de Carmichael
φ(n)=(p-1), isto é, d  é o inverso multiplicativo de e mod n, Zn é o grupo,
tal que n é o produto de dois números primos p e q e ambos de grande tamanho.
d- chave privada e (n, e)- chave pública.

Exemplo: Seja 128 o número de bits, suponhamos p=6076863405864212999 e q=1562809530454321421,
então o par de chaves é: chave privada d=4913970632284035618727648810595056111, chave pública:
(n, e)=(9496980045953699178562209452752351579, 404259412098621376441621593718777431).

Atenção: Se os parâmetros p e q forem de tamanhos pequenos, um ataque por força
é trivial. A NIST recomenda no mínimo 3072 bits.
    """
    p, q= next_prime(2**(bits/2)), next_prime(2**(bits))
    n1= p*q
    phi=(p-1)*(q-1)
    e=ZZ.random_element(phi)
    while gcd(e, phi) !=1:
        e=ZZ.random_element(phi)
    d= power_mod(e, -1, phi)
    Chaveprivada= d
    Chavepublica= (n1,e)
    return Chaveprivada, Chavepublica

# Ofuscação ALICE
def ofuscacao (m, Chavepublica):
    """
Função que invoca a ofuscação da mensagem antes da assinatura.
Input: mensagem e chave pública.
Output: constante de ofuscação e a mensagem ofuscada.

Uso: escolhe-se k pertencente a grupo Zn, e com k e a chave 
pública, o emissor calcula (k**e)*m (ofusca), e envia ao signatário
a mensagem ofuscada m0.

Exemplo: suponhamos que queremos ofuscar m= 70998000111870, então, m0=717349731693766832827349986846885114994760938826138214108
    """
    n1, e = Chavepublica
    Zn1=IntegerModRing(n1)
    k0=Zn1(randint(2,n1-1))
    m0=(k0**e)*m
    return (k0, m0)

# Momento da assinatura SIGNATÁRIO- CA
def assinaturacega (m0, Chaveprivada, Chavepublica):
    """
    Função que invoca a assinatura da mensagem.
    Input: mensagem ofuscada e o par de chave RSA.
    Output: assinatura da mensagem ofuscada.
    Uso: a assinatura é igual a potência cuja base é a mensagem ofuscada e
    o expoente é a chave privada
    """
    n1, _ = Chavepublica
    d = Chaveprivada
    sig =m0**d
    return sig

# Desofuscação ALICE
def assinaturalimpa (sig, k0, Chavepublica):
    """
Função que invoca a desofuscação do texto ofuscado.
Input: mensagem ofuscada ofuscada, factor de ofuscação e a
chave pública.
Output: texto claro assinado.

Uso: O emissor multiplica-se o texto ofuscado assinado pelo
inverso da constante de ofuscação.
    """
    n1, e = Chavepublica
    sigm = (sig)*(1/k0)
    return sigm

# ==================================================================================== PARTILHA

# Coeficientes do polinómio BOB
def Coefpolinomio (Zp, Prkey, n, k):
    """
Função que invoca a escolha dos coeficientes.
Input: recebe o primo p, o segredo, o número de participantes
e o número de elementos para recuperar.
Output: retorna os coeficientes do polinómio interpolador de Lagrange

Uso: o parâmetro a, corresponde aos k-1 números aleatórios.
    """
    a0=Prkey
    #Zp= IntegerModRing(p)
    t=k-1
    coef = [a0]
    for r in range (1, t+1):
        a = Zp.random_element()
        coef.append(a)
    return coef

# Construção do polinómio BOB
def polinomioconstruido (coef):
    """
Função que invoca a geração do polinómio interpolador de lagrange.
Input: recebe os coeficientes.
Output: retorna o polinómio.

Exemplo: suponhamos que temos o segredo 2000, k= 2. Escolhemos k-1=1
coeficiente [2000, 16749784847526677112774608782480510392], então o polinómio
é: P(x)= 16749784847526677112774608782480510392*x + 2000
    """
    Zp=coef[0].parent()
    Pol.<x> = PolynomialRing(Zp)
    polinomio=Pol(coef)
    return polinomio

# Partes à partilhar BOB
def partespartilhadas (n, polinomio):
    """
Função que invoca o cálculo das partes secretas.
Input: recebe n e o polinómio interpolador de Lagrange.
Output: retorna pares ordenados (1, polinomio(1)),...,(n, polinomio(n))

Uso: calcula-se as as imagens dos n números na função polinómial,
e retorna-se os pares ordenados secretos, e são distribuidos.

Exemplo: Seja bits= 128, suponhamos que queremos compartilhar 2000, as
partes secretas compartilhadas são:  [[1, 137624036525743829578287975035514517986], 
[2, 275248073051487659156575950071029033972], [3, 412872109577231488734863925106543549958]]
    """
    pares=[]
    for x in range (1, n+1):
        pares.append([x, polinomio(x)])
    return pares
# =========================================================================== VERIFICAÇÃO DA CONSISTÊNCIA

# Calculo das potencias BOB
i=0
pot=[]
def potencia (g, coef):
    """
Função que invoca o cálculo das potências
(parâmetros públicos) para a verificação.
Input: recebe o gerador e os coeficientes do polinómio.
Output: retorna as k potencias.

Uso: calcula-se e distribui-se as k potencias a todos os
participantes.
    """
    pot=[]
    k=len(coef)
    for i in range (k):
        pot.append(g**coef[i])
    return pot

# Verificação da consistência das partes secretas- MEC

def verificacaopartes (i, g, pot, parte_secreta):
    """
Função que invoca a verificação da consistência das partes
secretas de cada interveniente.
Input: recebe i-posição de cada participante, o gerador e as potencias.
Output: retorna dois valores, True ou False, caso positivo ou negativo, respetivamente.

Atenção: Se o resultado for False, implica dizer que, o responsável pela partilha
distribuiu maliciosamente a parte secreta ou por engano assim, a fez.
    """
    prod=1
    k=len(pot)
    for j in range (k):
        prod =prod*(pot[j]**(i**j))
    if g**parte_secreta == prod:
        return True
    else:
        return False

#RECONSTRUÇÃO DO POLINÓMIO- RE

def recostrucaopolinomio (coef, pares):
    """
Função que invoca a recuperação do segredo compartihado.
Input: recebe k e as partes secretas compartilhadas.
Output: retorna o polinómio interpolador.

Uso: calcula-se os polinómios de Langrange lj e em
seguida, calcula o polinómio interpolador pol.
então, a0 de pol é o segredo reconstruido.
    """
    Zp=coef[0].parent()
    Pol.<x> = PolynomialRing(Zp)
    k = len(coef)
    pol = Pol(0)
    for j in range(k):
        lj = Pol(Zp(1))
        for i in range(k):
            if j!=i:
                lj = lj * ((x-pares[i][0])/(pares[j][0] - pares[i][0]))
        pol = pol + lj * pares[j][1]
    return pol

# =====================================================================CIFRAÇÃO E DECIFRAÇÃO

#Cifração- ALICE
def cifracaovoto (PuKey, m):
    """
Função que invoca a cifração do texto puro.
Input: a chave pública e o texto puro.
Output: retorna o criptograma, e o inteiro escolhido por Alice.

Uso: Bob calcula o criptograma (α; β), efetuando,
α= g^k mod p, e β=M . y^k mod p, onde k1 é um inteiro pertecente
ao grupo.

Exemplo: Suponhamos que queremos cifrar M = 11002, então, o criptograma resultante é:
[218633365991801361628119371244493036581, 65561913107707291720633356647254684460]
    """
    p, g, y =PuKey
    Zp=IntegerModRing(p)
    k1=randint(2, p-1)
    Alfa=g**k
    Beta=m*(y**k)
    Criptograma=[Alfa, Beta]
    return Criptograma, k1

#Decifração- RE
def Decifracaovoto (PuKey, PrKey, Criptograma):
    """
Função que invoca a decifração do texto cifrado.
Input: a chave pública, privada e o criptograma.
Output: retorma a mensagem original.

Uso: Alice decifra a mensagem: Texto puro=M . α^-x . α^x mod p, sendo,
α= g^k recebido de Bob.
    """
    p, g, y = PuKey
    a0 = PrKey
    Zp=IntegerModRing(p)
    s=(Criptograma[0]**a0)
    t_p=(1/s)*Criptograma[1]
    return t_p
# =========================================================== VERIFICAÇÃO DA AUTENTICIDADE

# verificação da autenticidade- RE
def verificacaoassinatura (m, Chavepublica, sigm):
    """
Função que invoca a verificação da autenticidade.
Input: recebe a mensagem, a chave pública e a mensagem assinada.
Output: retorna dois valores lógicos: True-caso a auteticidade for válida e False-caso
a autenticidade for inválida.

Atenção: caso a autenticidade não for confirmada, o recetor rejeita a mensagem, isto,
a mensagem foi alvo de um ataque.
    """
    n1, e = Chavepublica
    if sigm**e==m:
        return True
    else:
        return False
# ===========================================================PROVA DE CONHECIMENTO ZERO

#Prova de conhecimento zero 

#INICIALIZAÇÃO
def parametropublico (bits):
    """
Função que invoca a inicialização da prova de conhecimento zero.
Input: recebe o número de bits.
Output: retorna o parâmetro público n.

Uso: escolhe-se p e q, númros e calcula-se n2= p . q.
    """
    p, q= random_prime(2**(bits/2)), random_prime(2**(bits/2))
    n2 = p*q
    return n2

#Escolha dos segredos
def secretsproof (n2):
    """
    Função que invoca a escolha de segredos.
    Input: parâmetro público n2.
    Output: retorna os segredos.
    Uso: Alice escolhe os números secretos pertencentes em Zn2.
    """
    Zn2 = IntegerModRing(n2)
    s1 = Zn2.random_element()
    s2= Zn2.random_element()
    s3 = Zn2.random_element()
    return s1, s2, s3

# Interação entre Paula e Veigas
def Comunicacaoproof (n2, s1, s2, s3):
    """
Função que invoca a interação entre o provador e o verificador.
Input: parâmetro público n2 e números secretos s1, s2, s3, ..., sk.
Onde k é um inteiro positivo.
Output: retorna r*(s1**a1)*(s2**a2)*(s3**a3), s*r**2 e a lista de
números binários, onde n pertence ao grupo Zn e r é 1 ou -1.

Uso: seja Zn2, ecolhe-se um número inteiro r e escolhe-se um número
s entre 1 ou -1. Calcula-se em seguida x=s.r**2, escolhe-se também
os números binários a1, a2, a3, ..., ak e calcula-se 
y=r.(s1^a1).(s2^a2).(s3^a3).

Exemplo: suponhamos n2 = 3117771557, s1= 12, s2=17, s3=99.
Paula interage com Veigas, enviando:1007592943, 1173404837, [1, 1, 0]

Atenção: a quantidade de números secretos e de números binários devem
ser iguais.
    """
    Zn2 = IntegerModRing(n2)
    r = Zn2.random_element()
    s = Zn2(1 - 2 * randint(0, 1))
    x = s*r**2
    a1 = randint(0, 1)
    a2 = randint(0, 1)
    a3 = randint(0, 1)
    y = r*(s1**a1)*(s2**a2)*(s3**a3)
    return y, x, [a1, a2, a3]

# Momento da prova
def Proof (n2, s1, s2, s3):
    """
Função que invoca a prova diante do verificador.
Input: Input: parâmetro público n2 e números secretos s1, s2, s3, ..., sk.
Onde k é um inteiro positivo.
Output: retorna os quadrados modulares v1, v2, v3, y, x e a lista dos 
números binários. Onde y e x são os dois primeiros parâmetros resultantes
da invocação da função Comunication.
    """
    Zn2=IntegerModRing(n2)
    v1 = Zn2(s1)**2
    v2 = Zn2(s2)**2
    v3 = Zn2(s3)**2
    comun = Comunicacaoproof (n2, s1, s2, s3)
    return (v1, v2, v3, comun[0:2]), comun[2]

# Verificação
def Verificacaoproof (proof, lista_a):
    """
Função que invoca a verificação da prova.
Input: receeb os elementos da prova: v1, v2, v3, y, x e os némeros binários.
Output: retorna dois valores, um True, outro False.

Uso: o verificador calcula w = -x*(v1**a1)*(v2**a2)*(v3**a3)
k = x*(v1**a1)*(v2**a2)*(v3**a3), e depois verifica se y^2 é igual a k ou w.
Se sim, Paula passa da prova, ao contrário, Paula reprova.
    """
    v1, v2, v3, (y, x) = proof
    [a1, a2, a3] = lista_a
    w = -x*(v1**a1)*(v2**a2)*(v3**a3)
    k = x*(v1**a1)*(v2**a2)*(v3**a3)
    if (y**2 == w or y**2 == k):
        return True
    else:
        return False

#========================================= INVOCANDO FUNÇÕES...
bits=128
pkEG, skEG, n, k, grupo, gerador=Elgamalinicial (bits)
m=voto()
m2=m-1 # .................................mani
sk, pk=chavesblindsig (bits)
#m=15
#m2=m-1 ..........................mani

#======================================== Assisnatura cega
(bfactor, bmess) = ofuscacao (m, pk)
bsig = assinaturacega (bmess, sk, pk)
sigmes = assinaturalimpa (bsig, bfactor, pk) #...... Assin

#=============================================verif da assinatura
ver2= verificacaoassinatura (m, pk, sigmes) #------------verif true

ver2_mani= verificacaoassinatura(m2, pk, sigmes) #------------verif false

#=============================================== Partilha
# n, k, g, p, secret = main(bits)
coef_pol=Coefpolinomio (grupo, skEG, n, k)
pol_const=polinomioconstruido (coef_pol)
partes=partespartilhadas (n, pol_const) #...... partes secretas
pol_reconst=recostrucaopolinomio (coef_pol, partes) #...... polinomio rec

#===================================================== Partilha mani
k_mani=k-1
coef_pol_mani=Coefpolinomio(grupo, skEG, n, k_mani)
pol_reconst_mani=recostrucaopolinomio(coef_pol_mani, partes)

#============================================ Feldman
ptt=potencia (gerador, coef_pol) #...... potencias distribuidas
t=2 #...... elemento a verificar
ver1=verificacaopartes (t, gerador, ptt,partes[t-1][1]) #...... verificacao da consis

#============================================ Feldman mani
sec=partes[t-1][1]+1
ver1_mani=verificacaopartes (t, gerador, ptt,sec)

#============================================ Cifracao/ decifracao
Criptograma, kapa1=cifracaovoto (pkEG, m)
texto_puro=Decifracaovoto (pkEG, skEG, Criptograma)

#============================================ Zero knowlodge
nn=parametropublico (bits)
sec1, sec2, sec3 = secretsproof (nn)
a, b, c=Comunicacaoproof (nn, sec1, sec2, sec3)
prova, binarios = Proof (nn, sec1, sec2, sec3)
verificacao3 = Verificacaoproof (prova, binarios)

#===================================================---Zero knowlodge mani
a, b, c= Comunicacaoproof (nn, sec1, sec2, sec3)
prova, binarios = Proof (nn, sec1, sec2, sec3)
#----------------------- manipulando os binarios
binarios_mani = [binarios[0]+1, binarios[1]+1, binarios[2]+1]
verificacao3_mani = Verificacaoproof (prova, binarios_mani)

#============================================= RESULTADOS DO ESQUEMA...


print("O(a) senhor(a) votou no candidato/ partido numero:", m)
print("")
print("")
print("O voto assinada e:", sigmes)
print("")
print("")
print("A chave privada COMPARTILHADA e:", skEG)
print("")
print("")
print("O polinomio interpolador construido e:", pol_const)
print("")
print("")
print("As partes secretas a compartilhar sao:", partes)
print("")
print("")
print("As potencias para a verificacao de consistencia sao:", ptt)
print("")
print("")
print("A verificacao da parte secreta distribuida ao PARTICIPANTE", t, "e:", ver1)
print("")
print("")
print("A verificacao da parte secreta distribuida ao PARTICIPANTE", t, "e:", ver1_mani)
print("")
print("")
print(" Temos", k, "partes secretas reunidas, e o segredo reconstruido e o termo independente do polinomio:", pol_reconst)
print("")
print("")
print(" Temos", k_mani, "partes secretas reunidas, e o segredo reconstruido e o termo independente do polinomio:", pol_reconst_mani)
print("")
print("")
print("O voto cifrado e:", Criptograma)
print("")
print("")
print("O voto decifrado e:", texto_puro)
print("")
print("")
print("A autencidade do voto e:", ver2)
print("")
print("")
print("A autencidade do voto e:", ver2_mani)
print("")
print("")
print("Bob escolhe os seguintes numeros para se comunicar com o provador:", binarios)
print("")
print("")
print("O verificador responde:", verificacao3)
print("")
print("")
print(" Os numeros que Bob escolheu para se comunicar com o provador foram alterados:", binarios_mani)
print("")
print("")
print("O verificador responde:", verificacao3_mani)

Quantos participantes terá o esquema de partilha da CHAVE PRIVADA?5
Quantas partes partes secretas serão necessárias para RECONSTRUIR A CHAVE?3
Digite um numero para eleger o seu CANDIDATO/ PARTIDO:20
('O(a) senhor(a) votou no candidato/ partido numero:', 20)


('O voto assinada e:', 5590531190346096141146496263558788022533255062872629584154)


('A chave privada COMPARTILHADA e:', 228998269188851665443735135331248134641)


('O polinomio interpolador construido e:', 89586562951068671314869182330534557263*x^2 + 87084561477128827105491793594341784573*x + 228998269188851665443735135331248134641)


('As partes secretas a compartilhar sao:', [[1, 405669393617049163864096111256124476477], [2, 761513643947384004914195451842069932839], [3, 1296531020179856188594033157089084503727], [4, 2010721522314465714903609226997168189141], [5, 2904085150351212583842923661566320989081]])


('As potencias para a verificacao de consistencia sao:', [39241265525578354159846414386997962684, 237725816016709938713